https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/

https://pubchem.ncbi.nlm.nih.gov/

https://chembl.gitbook.io/chembl-interface-documentation/downloads

https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/

In [2]:
!pip install chembl-webresource-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 531.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.7 MB/s eta 0:00:00


https://mybinder.org/v2/gh/chembl/chembl_webresource_client/master?filepath=demo_wrc.ipynb

In [4]:
from chembl_webresource_client.new_client import new_client

similarity = new_client.similarity
res = similarity.filter(smiles="CO[C@@H](CCC#C\C=C/CCCC(C)CCCCC=C)C(=O)[O-]", similarity=70).only(['molecule_chembl_id', 'similarity'])
for i in res:
    print(i)

{'molecule_chembl_id': 'CHEMBL478779', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL477889', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL477888', 'similarity': '85.4166686534881591796875'}
{'molecule_chembl_id': 'CHEMBL2304268', 'similarity': '70.1754391193389892578125'}


In [5]:
from chembl_webresource_client.new_client import new_client

def perform_similarity_search(smiles, threshold=80):
    similarity = new_client.similarity
    # Perform similarity search using ChEMBL's similarity search API
    res = similarity.filter(smiles=smiles, similarity=threshold).only(['molecule_chembl_id', 'similarity'])
    similar_compounds = [(entry['molecule_chembl_id'],round(float(entry['similarity']),2)) for entry in res]
    similar_compounds_ = [entry['molecule_chembl_id'] for entry in res]
    return similar_compounds_

# Screening compounds with their IDs and SMILES strings
screening_compounds = {
    'NCGC00125704': 'CC1=CC=C(C=C1)N2C(=C3C(=NN(C(=O)C3=N2)CCCC(=O)NCC4=C(C(=CC=C4)OC)OC)C)C'}

#    'NCGC00602080-03': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=C(F)C=CC=C4)N=C2C',
 #   'NCGC00685598-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC=C4)N=C2C',
  #  'NCGC00685517-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC=CC(F)=C4)N=C2C',
   # 'NCGC00686269-01': 'CC1=C2C(=NN1C3=CC=C(C)C=C3)C(=O)N(CCCCNCCC4=CC(F)=CC(F)=C4)N=C2C'}

#NCGC00125704.
#Pubchem SID 24401087
#Compound CID: 16022698


# Perform chemical structure similarity search against ChEMBL for each screening compound
similar_compounds_dict = {}
for compound_id, compound_smiles in screening_compounds.items():
    similar_compounds = perform_similarity_search(compound_smiles)
    print(similar_compounds)
    similar_compounds_dict[compound_id] = similar_compounds

similar_compounds_dict

['CHEMBL1372162', 'CHEMBL1367890', 'CHEMBL1733809']


{'NCGC00125704': ['CHEMBL1372162', 'CHEMBL1367890', 'CHEMBL1733809']}

In [3]:
from chembl_webresource_client.new_client import new_client

def perform_similarity_search(cid, threshold=80):
    similarity = new_client.similarity
    # Perform similarity search using ChEMBL's similarity search API
    res = similarity.filter(target_chembl_id=cid, similarity=threshold).only(['molecule_chembl_id', 'similarity'])
    similar_compounds = [(entry['molecule_chembl_id'], round(float(entry['similarity']), 2)) for entry in res]
    similar_compounds_ = [entry['molecule_chembl_id'] for entry in res]
    return similar_compounds_

# Screening compounds with their IDs and CID
screening_compounds = {
    'NCGC00125704': '16022698'}

# Perform chemical structure similarity search against ChEMBL for each screening compound
similar_compounds_dict = {}
for compound_id, compound_cid in screening_compounds.items():
    similar_compounds = perform_similarity_search(compound_cid)
    print(similar_compounds)
    similar_compounds_dict[compound_id] = similar_compounds

print(similar_compounds_dict)

HttpBadRequest: Error for url https://www.ebi.ac.uk/chembl/api/data/similarity.json, server response: {"error_message": "Structure or identifier required."}

In [6]:
from chembl_webresource_client.new_client import new_client

# Function to retrieve targets associated with a ChEMBL ID
def get_gene_symbols(entry):
    gene_symbols = []
    for component in entry['target_components']:
        for synonym in component['target_component_synonyms']:
            #if synonym['syn_type'] in ['GENE_SYMBOL', 'GENE_SYMBOL_OTHER']:
                gene_symbols.append(synonym['component_synonym'])
    return tuple(gene_symbols)  # Convert list to tuple

def get_targets(chembl_id):
    target_client = new_client.target
    # Retrieve targets associated with the ChEMBL ID
    res = target_client.filter(target=chembl_id,organism='Homo sapiens').only(['target_chembl_id','organism','pref_name','target_components'])
    #res = res[:10]
    # Extract the target ChEMBL IDs
    return {(entry['organism'], entry['target_chembl_id'], entry['pref_name'], get_gene_symbols(entry)) for entry in res}

    #return {(entry['organism'],entry['target_chembl_id'], entry['pref_name']) for entry in res}


# Function to find shared targets between two ChEMBL IDs
def find_shared_targets(chembl_id1, chembl_ids):
    # Retrieve targets associated with each ChEMBL ID
    targets1 = get_targets(chembl_id1)
    # Find the intersection of targets to get shared targets
    shared_targets = set()
    for chembl_id in chembl_ids:
        targets2 = get_targets(chembl_id)
        shared_targets.update(targets1.intersection(targets2))
    return shared_targets

# Get targets associated with ChEMBL compounds
chembl_targets = set()
for similar_compounds in similar_compounds_dict.values():
    chembl_targets.update(similar_compounds)

TOR1A = 'CHEMBL1829'  # Define the TOR1A target

# Find shared targets
shared_targets = find_shared_targets(TOR1A, chembl_targets)
print("Shared Targets:", list(shared_targets))  # Convert set to list for printing

Shared Targets: [('Homo sapiens', 'CHEMBL2366293', 'ST486', ()), ('Homo sapiens', 'CHEMBL4523509', 'Spindlin-1', ('OCR', 'Ovarian cancer-related protein', 'SPIN', 'SPIN1', 'Spindlin1', 'Spindlin-1')), ('Homo sapiens', 'CHEMBL614517', 'A673', ()), ('Homo sapiens', 'CHEMBL1075579', 'SK-N-DZ', ()), ('Homo sapiens', 'CHEMBL5870', 'Centromere-associated protein E', ('CENPE', 'CENP-E', 'Centromere-associated protein E', 'Centromere protein E', 'Kinesin-7', 'Kinesin-related protein CENPE')), ('Homo sapiens', 'CHEMBL1075439', 'EFM-19', ()), ('Homo sapiens', 'CHEMBL614734', 'HMEC', ()), ('Homo sapiens', 'CHEMBL2363068', 'Prostaglandin E2 receptor', ('PGE2 receptor EP1 subtype', 'PGE receptor EP1 subtype', 'Prostaglandin E2 receptor EP1 subtype', 'Prostanoid EP1 receptor', 'PTGER1', 'PGE2 receptor EP4 subtype', 'PGE receptor EP4 subtype', 'Prostaglandin E2 receptor EP4 subtype', 'Prostanoid EP4 receptor', 'PTGER2', 'PTGER4', 'PGE2 receptor EP2 subtype', 'PGE receptor EP2 subtype', 'Prostaglandin

In [7]:
species=[]
for i in shared_targets:
  species.append(i[0])
list(set(species))

['Homo sapiens']

In [8]:
symbols=[]
for i in shared_targets:
  symbols+=i[3]
list(set(symbols))

['PLEC',
 'CPD',
 'Annexin-2',
 'CAK-beta',
 'TFCOUP1',
 'MAP3K11',
 'MTND6',
 'PSP24-beta',
 'Terminal addition enzyme',
 'OPRM1',
 'K86',
 'CCP1',
 'Deubiquitinating enzyme 30',
 'GPR74',
 'Apo-2 ligand',
 'Aldehyde dehydrogenase 7',
 'Peptidase M 1',
 'Organic anion-transporting polypeptide 1',
 'NUDIX5',
 'PIP5K1A',
 'LXA4 receptor',
 'IKAROS',
 'Tho4',
 'IFN-alpha-10',
 'Thioredoxin-dependent peroxide reductase A0372',
 'MET1',
 'Ubiquitin thioesterase OTU1',
 'CTCFL paralog',
 'CI-MLRQ',
 '60S ribosomal protein L7a',
 'KIAA0070',
 'Cystatin-C',
 'Yolk sac permease-like molecule 2',
 'hPC6',
 'S1PR3',
 'GPR135',
 'B lymphocyte serine/threonine-protein kinase',
 'Alpha-2 type I collagen',
 '2.4.2.-',
 'UDP-glucuronosyltransferase 2A3',
 'EPHA3',
 'MLL3',
 'NFAT transcription complex cytosolic component',
 '60S ribosomal protein L26',
 'Tumor necrosis factor superfamily member 5-induced protein 1',
 'Squamous cell carcinoma-related oncogene 5',
 'GRIM12',
 'INO80 complex subunit J',

In [ ]:
for entry in res:
      X[ (entry['organism'],entry['target_chembl_id'], entry['pref_name'],AA)   ]=None
      return X

In [ ]:
# Function to retrieve targets associated with a ChEMBL ID
def get_targets(chembl_id):
    target_client = new_client.target
    # Retrieve targets associated with the ChEMBL ID
    res = target_client.filter(target=chembl_id).only(['target_chembl_id','organism','pref_name','target_components'])
    res = res[:100]
    # Extract the target ChEMBL IDs
    X=[]

    return {(entry['organism'],entry['target_chembl_id'], entry['pref_name']) for entry in res}



TOR1A = 'CHEMBL1829'  # Define the TOR1A target
get_targets(TOR1A)

{('Ascaris suum',
  'CHEMBL1884',
  'Nicotinic acetylcholine receptor alpha subunit'),
 ('Candida albicans (strain SC5314 / ATCC MYA-2876) (Yeast)',
  'CHEMBL1780',
  'Cytochrome P450 51'),
 ('Escherichia coli', 'CHEMBL1809', 'Dihydrofolate reductase'),
 ('Escherichia coli', 'CHEMBL1826', 'DNA gyrase subunit B'),
 ('Escherichia coli', 'CHEMBL2065', 'Beta-lactamase TEM'),
 ('Escherichia coli K-12', 'CHEMBL1813', 'Penicillin-binding protein 1A'),
 ('Escherichia coli K-12', 'CHEMBL1840', 'Penicillin-binding protein 2'),
 ('Escherichia coli K-12',
  'CHEMBL1852',
  'DNA-directed RNA polymerase beta chain'),
 ('Escherichia coli K-12',
  'CHEMBL1857',
  'Enoyl-[acyl-carrier-protein] reductase'),
 ('Escherichia coli K-12', 'CHEMBL1858', 'DNA gyrase subunit A'),
 ('Escherichia coli K-12', 'CHEMBL1956', 'D-alanylalanine synthetase'),
 ('Escherichia coli K-12',
  'CHEMBL1984',
  'UDP-N-acetylglucosamine 1-carboxyvinyltransferase'),
 ('Herpes simplex virus (type 1 / strain 17)',
  'CHEMBL1872',
 

https://hpo.jax.org/app/browse/disease/ORPHA:98807

https://hpo.jax.org/app/browse/gene/1861

In [ ]:
https://hpo.jax.org/app/browse/disease/ORPHA:98807

#####################################################################################

Human Phenotype Ontology (HPO): Utilize the Human Phenotype Ontology, which provides standardized vocabulary for describing phenotypic abnormalities associated with human diseases. You can query the HPO to identify phenotypes specifically linked to DYT1 dystonia.

Phenotypic Annotations: Retrieve phenotypic annotations associated with TOR1A (TorsinA) and the targets associated with the screening compounds from databases or literature sources. These annotations describe the phenotypic abnormalities observed in individuals with mutations or alterations in these genes.

Comparison: Compare the phenotypic annotations of TOR1A with those of the targets associated with the screening compounds. Look for shared phenotypes that are specifically associated with DYT1 dystonia. Phenotypes such as "involuntary movements," "muscle spasms," "dystonia," and "motor dysfunction" may be relevant in this context.

Relevance Assessment: Assess the relevance of the shared phenotypes to DYT1 dystonia. Phenotypes that are closely related to the clinical manifestations of the disease provide evidence of potential relevance of the targets to DYT1 dystonia.

Prioritization: Prioritize the targets based on the extent of shared phenotypes with TOR1A and the severity of the associated phenotypic abnormalities. Targets showing a high degree of overlap with TOR1A and exhibiting clinically relevant phenotypes associated with DYT1 dystonia may represent promising candidates for further investigation.

By integrating phenotypic comparisons using HPO annotations, we can enhance your understanding of the potential relevance of the identified targets to DYT1 dystonia and prioritize them for further experimental validation and drug discovery efforts. This approach strengthens the translational relevance of your findings and facilitates the identification of novel therapeutic targets for the disease.

In [1]:
import json
import pandas as pd

# Read the JSON file
file_path = '/content/hp.json'
with open(file_path, 'r') as file:
    data = json.load(file)
# Extract relevant information from the JSON dat
graphs = data['graphs']
nodes = graphs[0]['nodes']  # Assuming there's only one graph in the JSON data
# Initialize lists to store data
ids = []
labels = []
types = []
# Iterate over the nodes and extract the required information
for node in nodes:
    ids.append(node['id'])
    labels.append(node.get('lbl', ''))
    types.append(node.get('type', ''))
# Create a DataFrame from the extracted data
HP = pd.DataFrame({
    'id': ids,
    'label': labels,
    'type': types
})

import pandas as pd
phenotype_to_genes = pd.read_csv('/content/phenotype_to_genes.txt', sep='\t')
genes_to_phenotype = pd.read_csv('/content/genes_to_phenotype.txt', sep='\t')
genes_to_disease = pd.read_csv('/content/genes_to_disease.txt', sep='\t')
phenotype = pd.read_csv('/content/phenotype.hpoa', sep='\t', skiprows=4)


<ipython-input-1-781703eb0002>:31: DtypeWarning: Columns (2,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  phenotype = pd.read_csv('/content/phenotype.hpoa', sep='\t', skiprows=4)


In [ ]:
Shared Targets:
('Homo sapiens', 'CHEMBL1843', 'DOPA decarboxylase', ('AADC', 'DDC', 'Synonyms=AADC’)),
 ('Escherichia coli', 'CHEMBL1809', 'Dihydrofolate reductase', ('folA', 'tmrA ‘)),
 ('Homo sapiens', 'CHEMBL1952', 'Thymidylate synthase', ('TS', 'TYMS’)),
 ('Homo sapiens', 'CHEMBL222', 'Norepinephrine transporter', ('NAT1', 'NET1', 'SLC6A2', 'SLC6A5’)),
 ('Homo sapiens', 'CHEMBL2033', 'Cytochrome P450 11A1', ('CYP11A', 'CYP11A1', 'Synonyms=CYP11A’)),
 ('Homo sapiens', 'CHEMBL211', 'Muscarinic acetylcholine receptor M2', ('CHRM2',)),
 ('Homo sapiens', 'CHEMBL1968', 'Epoxide hydrolase 1', ('EPHX', 'EPHX1', 'EPOX’)),
 And etc.


In [11]:
Shared_gene=[]
for i in list(shared_targets):
  Shared_gene+=list((i[3]))
Shared_gene=list(set(Shared_gene))
Shared_gene[:4]


['PLEC', 'CPD', 'Annexin-2', 'CAK-beta']

In [12]:
shared_gene_phenotype_table=pd.DataFrame()
shared_gene_phenotype_table['shared_gene']=Shared_gene
shared_gene_phenotype_table.to_csv('shared_gene_phenotype_table.csv', index=False)

In [61]:
import pandas as pd
shared_gene_phenotype_table = pd.read_csv('/content/shared_gene_phenotype_table.csv')
genes_to_phenotype = pd.read_csv('/content/genes_to_phenotype.txt', sep='\t')
# Preprocess genes_to_phenotype DataFrame
gene_to_hpo_dict = genes_to_phenotype.groupby('gene_symbol').agg({'hpo_name': list}).to_dict()['hpo_name']
gene_to_hpo_dict_lower = {key.lower(): value for key, value in gene_to_hpo_dict.items()}
# Modify the function to use the preprocessed dictionary
def finding_hpo(x):
    return gene_to_hpo_dict_lower.get(x.lower(), [])

# Apply the function
shared_gene_phenotype_table['phenotype'] = shared_gene_phenotype_table['shared_gene'].str.lower().apply(finding_hpo)
#shared_gene_phenotype_table['phenotype'] = shared_gene_phenotype_table.apply(lambda x: x['phenotype'] + [x['shared_gene']], axis=1)
shared_gene_phenotype_table['shared_gene'] = shared_gene_phenotype_table.apply(lambda x:  [x['shared_gene']], axis=1)

shared_gene_phenotype_table

,shared_gene,phenotype
0,[PLEC],"[Microtia, Microtia, Pelvic girdle muscle weak..."
1,[CPD],[]
2,[Annexin-2],[]
3,[CAK-beta],[]
4,[TFCOUP1],[]
...,...,...
33902,[KIFC3],[]
33903,[F9],"[Deep venous thrombosis, Osteoarthritis, X-lin..."
33904,[BBAP],[]
33905,[Alpha-atrial natriuretic peptide],[]


In [37]:
shared_gene_phenotype_table.to_csv('shared_gene_phenotype_table.csv', index=False)

# prioritization based on literature mining

In [35]:
Lit_mining=pd.read_csv('/content/dyt1.csv')


import ast

def extract_names_from_string(txt_str):
    # Convert the string representation of the list to an actual list
    txt = ast.literal_eval(txt_str)
    unique_names = set()
    for sublist in txt:
        for name in sublist:
          if name.lower() not in ['dyt1','tor1a','primary dystonias'  ]:
            unique_names.add(name.strip())
    return list(unique_names)


Lit_mining['related_gene']=Lit_mining.apply(lambda x:  extract_names_from_string(x['texts']),axis=1)
display(Lit_mining['related_gene'])

0      [parkinsonism, myoclonus, X-linked dystonia-pa...
1      [isolated dystonias, myoclonus, DYT24, Movemen...
2      [REEP4, c.629delC, c.192G > A, R149Q, rs143556...
3      [dystonic tremor, parkinsonism, myoclonus, DYT...
4      [mice, torsinA, HVA, norepinephrine, hMT2, hom...
                             ...                        
380                                   [torsion dystonia]
381                        [idiopathic torsion dystonia]
382                         [primary torsional dystonia]
383                                   [primary dystonia]
384                                                   []
Name: related_gene, Length: 385, dtype: object

### new method

In [62]:
shared_gene_phenotype_table['count_phenotype'] = 0
shared_gene_phenotype_table['count_gene'] = 0
shared_gene_phenotype_table['count']=0
shared_gene_phenotype_table

,shared_gene,phenotype,count_phenotype,count_gene
0,[PLEC],"[Microtia, Microtia, Pelvic girdle muscle weak...",0,0
1,[CPD],[],0,0
2,[Annexin-2],[],0,0
3,[CAK-beta],[],0,0
4,[TFCOUP1],[],0,0
...,...,...,...,...
33902,[KIFC3],[],0,0
33903,[F9],"[Deep venous thrombosis, Osteoarthritis, X-lin...",0,0
33904,[BBAP],[],0,0
33905,[Alpha-atrial natriuretic peptide],[],0,0


In [ ]:
nltk.download('punkt')
def get_stem(text):
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Get the stem of each word and join them back into a sentence
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [63]:
import re
list_tor1a = ['dyt1', 'tor1a', 'primary dystonias', 'DYT1 dystonia', 'TOR1A dystonia',
                  'Early-onset torsion dystonia', "Oppenheim's dystonia", 'Primary torsion dystonia',
                  'Idiopathic torsion dystonia']

def calculate_count(shared_gene_phenotype_table, Lit_mining):
    # Initialize count column to zeros
    #shared_gene_phenotype_table['count'] = 0
    # Compile a regex pattern to split text into sentences
    sentence_splitter = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    # Iterate over rows in shared_gene_phenotype_table

        # Iterate over rows in Lit_mining
    for _, lit_row in Lit_mining.iterrows():
        abstract_text = lit_row['a.abstractText'].lower()
        sentences =[lit_row['a.title'].lower()]+ sentence_splitter.split(abstract_text)
        for index, row in shared_gene_phenotype_table.iterrows():
            phenotype_values = row['phenotype']
            # Check for concurrency with phenotype_values and list_tor1a
            if any(any(phenotype.lower() in sentence and any(tor1a.lower() in sentence for tor1a in list_tor1a) for phenotype in phenotype_values) for sentence in sentences):
                shared_gene_phenotype_table.at[index, 'count_phenotype'] += 1
                #break  # Move to the next row in shared_gene_phenotype_table once count is updated

    return shared_gene_phenotype_table
#Lit_mining1=Lit_mining[Lit_mining['a.abstractText']!='""']
shared_gene_phenotype_table1=shared_gene_phenotype_table# [shared_gene_phenotype_table['phenotype']!= []]
shared_gene_phenotype_table1= calculate_count(shared_gene_phenotype_table1, Lit_mining)
shared_gene_phenotype_table1

,shared_gene,phenotype,count_phenotype,count_gene,count
0,[PLEC],"[Microtia, Microtia, Pelvic girdle muscle weak...",27,0,0
1,[CPD],[],0,0,0
2,[Annexin-2],[],0,0,0
3,[CAK-beta],[],0,0,0
4,[TFCOUP1],[],0,0,0
...,...,...,...,...,...
33902,[KIFC3],[],0,0,0
33903,[F9],"[Deep venous thrombosis, Osteoarthritis, X-lin...",0,0,0
33904,[BBAP],[],0,0,0
33905,[Alpha-atrial natriuretic peptide],[],0,0,0


In [64]:
import re
list_tor1a = ['dyt1', 'tor1a', 'primary dystonias', 'DYT1 dystonia', 'TOR1A dystonia',
                  'Early-onset torsion dystonia', "Oppenheim's dystonia", 'Primary torsion dystonia',
                  'Idiopathic torsion dystonia']

def calculate_count(shared_gene_phenotype_table, Lit_mining):
    # Initialize count column to zeros
    # Compile a regex pattern to split text into sentences
    sentence_splitter = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    # Iterate over rows in shared_gene_phenotype_table

        # Iterate over rows in Lit_mining
    for _, lit_row in Lit_mining.iterrows():
        abstract_text = lit_row['a.abstractText'].lower()
        sentences =[lit_row['a.title'].lower()]+ sentence_splitter.split(abstract_text)
        for index, row in shared_gene_phenotype_table.iterrows():
            phenotype_values = row['shared_gene']
            # Check for concurrency with phenotype_values and list_tor1a
            if any(any(phenotype.lower() == word and any(tor1a.lower() == word for tor1a in list_tor1a) for word in sentence.split()) for phenotype in phenotype_values for sentence in sentences):
                shared_gene_phenotype_table.at[index, 'count_gene'] += 1
                #break  # Move to the next row in shared_gene_phenotype_table once count is updated

    return shared_gene_phenotype_table
#Lit_mining1=Lit_mining[Lit_mining['a.abstractText']!='""']
shared_gene_phenotype_table1= calculate_count(shared_gene_phenotype_table1, Lit_mining)
shared_gene_phenotype_table1['count']=shared_gene_phenotype_table1['count_phenotype'] + shared_gene_phenotype_table1['count_gene']

shared_gene_phenotype_table1

,shared_gene,phenotype,count_phenotype,count_gene,count
0,[PLEC],"[Microtia, Microtia, Pelvic girdle muscle weak...",27,0,0
1,[CPD],[],0,0,0
2,[Annexin-2],[],0,0,0
3,[CAK-beta],[],0,0,0
4,[TFCOUP1],[],0,0,0
...,...,...,...,...,...
33902,[KIFC3],[],0,0,0
33903,[F9],"[Deep venous thrombosis, Osteoarthritis, X-lin...",0,0,0
33904,[BBAP],[],0,0,0
33905,[Alpha-atrial natriuretic peptide],[],0,0,0


In [65]:
shared_gene_phenotype_table1['count']=shared_gene_phenotype_table1['count_phenotype'] + shared_gene_phenotype_table1['count_gene']
shared_gene_phenotype_table1

,shared_gene,phenotype,count_phenotype,count_gene,count
0,[PLEC],"[Microtia, Microtia, Pelvic girdle muscle weak...",27,0,27
1,[CPD],[],0,0,0
2,[Annexin-2],[],0,0,0
3,[CAK-beta],[],0,0,0
4,[TFCOUP1],[],0,0,0
...,...,...,...,...,...
33902,[KIFC3],[],0,0,0
33903,[F9],"[Deep venous thrombosis, Osteoarthritis, X-lin...",0,0,0
33904,[BBAP],[],0,0,0
33905,[Alpha-atrial natriuretic peptide],[],0,0,0


In [66]:

shared_gene_phenotype_table1.to_csv('shared_gene_phenotype_table.csv', index=False)

### old method

In [30]:
def count_occurrences(df, column_name):
    occurrences = {}
    for lst in df[column_name]:
        for value in lst:
            if value in occurrences:
                occurrences[value.lower()] += 1
            else:
                occurrences[value.lower()] = 1
    return occurrences

occurrences_dict = count_occurrences(Lit_mining, 'related_gene')
display(occurrences_dict)

0      [DYT5a, TAF1, movement disorders, dystonias, D...
1      [DYT5a, isolated dystonias, TAF1, dystonias, D...
2      [rs1435566780, c.649C > T, G > A, K64K, rs5459...
3      [Dystonia, dystonic tremor, TAF1, DYT8, DYT23,...
4      [mouse, human, serotonin, 5-hydroxyindoleaceti...
                             ...                        
380                                   [torsion dystonia]
381                        [idiopathic torsion dystonia]
382                         [primary torsional dystonia]
383                                   [primary dystonia]
384                                                   []
Name: related_gene, Length: 385, dtype: object

{'dyt5a': 1,
 'taf1': 1,
 'movement disorders': 1,
 'dystonias': 40,
 'dyt8': 1,
 'seizures': 2,
 'sgce': 1,
 'parkinsonism': 1,
 'dystonia': 20,
 'movement abnormalities': 1,
 'thap1': 1,
 'dyt5b': 1,
 'dystonia plus syndromes': 3,
 'prkra': 1,
 'x-linked dystonia-parkinsonism syndrome': 1,
 'dystonias and dystonia plus syndromes': 1,
 'dyt12': 1,
 'atp1a3': 1,
 'gch1': 1,
 'dyt18': 1,
 'dyt11': 1,
 'dyt6': 1,
 'l-dopa': 1,
 'tremor': 24,
 'dyt10': 1,
 'myoclonus': 19,
 'slc2a1': 1,
 'patients': 74,
 'dyt3': 1,
 'dyt14': 1,
 'paroxysmal dystonias': 2,
 'movement anomalies': 1,
 'dyt9': 1,
 'dyt16': 1,
 'dyskinesias': 1,
 'isolated dystonias': 1,
 'dyt23': 1,
 'hyperkinetic movement disorders': 4,
 'tubb4': 1,
 'mr-1': 1,
 'ano3': 1,
 'dyt25': 1,
 'dyt4': 1,
 'ciz1': 1,
 'gtp cyclohydrolase i': 1,
 'dyt24': 1,
 'prrt2': 1,
 'gnal': 1,
 'dopa-responsive dystonia': 18,
 'segmental dystonia': 6,
 'rs1435566780': 1,
 'c.649c > t': 1,
 'g > a': 1,
 'k64k': 1,
 'rs545930392': 1,
 'eyelid spa

In [38]:
prioritized_gene=pd.DataFrame()
shared_gene_lower=[i.lower()for i in Shared_gene]
n=0
for i in shared_gene_lower:
   if i in occurrences_dict:
       prioritized_gene.at[n,'gene']= i
       prioritized_gene.at[n,'concurrency']= occurrences_dict[i]
       n+=1
sorted_prioritized_gene = prioritized_gene.sort_values(by='concurrency', ascending=False)
sorted_prioritized_gene.index=range(sorted_prioritized_gene.shape[0])
sorted_prioritized_gene

,gene,concurrency
0,pts,114.0
1,gelsolin,3.0
2,vesicular acetylcholine transporter,3.0
3,tau,2.0
4,vimentin,2.0
...,...,...
93,atp1a3,1.0
94,cog4,1.0
95,plectin,1.0
96,col6a3,1.0


In [47]:
sorted_prioritized_gene[:8]

,gene,concurrency
0,pts,114.0
1,gelsolin,3.0
2,vesicular acetylcholine transporter,3.0
3,tau,2.0
4,vimentin,2.0
5,gnao1,1.0
6,acat1,1.0
7,arsg,1.0


#hpo

In [56]:
Shared_gene= list(sorted_prioritized_gene[sorted_prioritized_gene['concurrency'] > 1]['gene'].values)
Shared_gene

['pts', 'gelsolin', 'vesicular acetylcholine transporter', 'tau', 'vimentin']

In [57]:
Shared_hpo=[]
list_gene_symbol=list(genes_to_phenotype['gene_symbol'])
list_gene_symbol=[i.lower() for i in list_gene_symbol]
for i in Shared_gene:
  if i in list_gene_symbol:
     Shared_hpo.append(i)
Shared_hpo[:7]

['pts']

In [60]:
hpo_names=[]
for i in Shared_hpo:
    hpo_names+=list(genes_to_phenotype[genes_to_phenotype['gene_symbol'].str.lower()==i]['hpo_name'])
hpo_names=list(set(hpo_names))
hpo_names

['Dystonia',
 'Clonus',
 'Motor delay',
 'Limb hypertonia',
 'Oculogyric crisis',
 'Bradykinesia',
 'Restlessness',
 'Excessive salivation',
 'Excessive daytime somnolence',
 'Neonatal onset',
 'Depression',
 'Agitation',
 'Pallor',
 'Poor head control',
 'Dysphagia',
 'Poor suck',
 'Global developmental delay',
 'Irritability',
 'Feeding difficulties',
 'Myoclonus',
 'Rigidity',
 'Progressive neurologic deterioration',
 'Hypsarrhythmia',
 'Hyperphenylalaninemia',
 'Parkinsonism',
 'Decreased CSF homovanillic acid concentration',
 'Infantile onset',
 'Autosomal recessive inheritance',
 'Hypertonia',
 'Intellectual disability',
 'Choreoathetosis',
 'Opisthotonus',
 'Delayed speech and language development',
 'Microcephaly',
 'Hypotonia',
 'Chorea',
 'Decreased CSF 5-hydroxyindolacetic acid concentration',
 'Recurrent fever',
 'Small for gestational age',
 'Falls',
 'Tremor',
 'Seizure',
 'Intellectual disability, progressive',
 'Hyperreflexia',
 'Hyperkinetic movements',
 'Ptosis',
 'Ab

In [61]:
genes_to_phenotype[genes_to_phenotype['gene_symbol'].str.lower()=='pts']#'TOR1A']

#'Homo sapiens', 'CHEMBL217', 'Dopamine D2 receptor'

,ncbi_gene_id,gene_symbol,hpo_id,hpo_name,frequency,disease_id
121910,5805,PTS,HP:0002487,Hyperkinetic movements,HP:0040283,ORPHA:13
121911,5805,PTS,HP:0003785,Decreased CSF homovanillic acid concentration,1/2,OMIM:261640
121912,5805,PTS,HP:0003781,Excessive salivation,HP:0040283,ORPHA:13
121913,5805,PTS,HP:0003781,Excessive salivation,-,OMIM:261640
121914,5805,PTS,HP:0002421,Poor head control,HP:0040283,ORPHA:13
...,...,...,...,...,...,...
121966,5805,PTS,HP:0000713,Agitation,HP:0040283,ORPHA:13
121967,5805,PTS,HP:0000980,Pallor,HP:0040283,ORPHA:13
121968,5805,PTS,HP:0000252,Microcephaly,-,OMIM:261640
121969,5805,PTS,HP:0001518,Small for gestational age,1/2,OMIM:261640


We calculate the count of shared phenotypes between each target and TOR1A (ENSG00000164087) and store them in a dictionary shared_phenotypes_count.

We define relevance scores for each target based on domain knowledge.

We calculate a score for each target by multiplying the count of shared phenotypes by its relevance score.

We sort the targets based on their scores in descending order to prioritize them. The higher the score, the higher the priority. You can adjust the relevance scores based on your understanding of the targets' importance to DYT1 dystonia.

In [ ]:
# Find shared phenotypes count for each target
shared_phenotypes_count = find_shared_phenotypes('ENSG00000164087', ensembl_ids)  # Example Ensembl ID for TOR1A
print("Shared Phenotypes Count:", shared_phenotypes_count)  # Print shared phenotypes count


# Define relevance scores for each target (you can modify these scores based on domain knowledge)
relevance_scores = {
    'ENSG00000164109': 5,  # Example relevance score for target 1
    'ENSG00000139618': 3,  # Example relevance score for target 2
    'ENSG00000204498': 2   # Example relevance score for target 3
}

# Prioritize targets based on shared phenotypes count and relevance to DYT1 dystonia
target_scores = {ensembl_id: shared_phenotypes_count[ensembl_id] * relevance_scores[ensembl_id] for ensembl_id in ensembl_ids}
# Sort targets based on scores in descending order
sorted_targets = sorted(target_scores.items(), key=lambda x: x[1], reverse=True)
print("Sorted Targets (Ensembl ID, Score):", sorted_targets)  # Print sorted targets

Shared Phenotypes Count: {'ENSG00000164109': 0, 'ENSG00000139618': 0, 'ENSG00000204498': 0}
Sorted Targets (Ensembl ID, Score): [('ENSG00000164109', 0), ('ENSG00000139618', 0), ('ENSG00000204498', 0)]


Here are some additional steps you can consider:

Target Validation: Verify the relevance of the identified targets in the context of DYT1 dystonia. Are these targets known to be involved in the disease pathway? Are there any literature references supporting their role in DYT1 dystonia?

Pathway Analysis: Investigate the biological pathways associated with the identified targets. Are there common pathways or biological processes shared among these targets? Are any of these pathways relevant to the pathogenesis of DYT1 dystonia?

Structural Analysis: Examine the three-dimensional structures of the identified targets and the binding sites of the screening compounds. Are there structural features that suggest potential interactions between the compounds and the targets? Can computational docking studies be performed to predict binding affinities and modes of interaction?

Functional Assays: Consider conducting functional assays to validate the effects of the screening compounds on the identified targets. This could include in vitro assays to assess protein activity, cellular assays to evaluate phenotypic changes, or animal models to study therapeutic efficacy in vivo.

Lead Optimization: If promising compounds are identified, explore strategies for lead optimization to improve potency, selectivity, and pharmacokinetic properties. This could involve medicinal chemistry efforts to design analogs with improved drug-like properties.

By integrating these additional analyses and experimental approaches, you can further elucidate the potential mechanisms of action of the screening compounds and advance towards the identification of novel therapeutic candidates for DYT1 dystonia.

# Pathway analysis

 investigate the biological pathways associated with the identified targets, you can use pathway databases such as Reactome, KEGG, or WikiPathways. Below is an example of how you can perform pathway analysis using the Reactome database:

In [ ]:
import requests

# Function to retrieve pathways associated with a given gene (Ensembl ID)
def get_pathways(ensembl_id):
    # Construct the URL for Reactome pathway analysis
    url = f"https://reactome.org/ContentService/data/pathways/low/entity/{ensembl_id}"
    # Send GET request to Reactome API
    response = requests.get(url)
    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Extract pathway information
        pathways = [pathway['displayName'] for pathway in data]
        return pathways
    else:
        print("Error:", response.text)
        return []

# Example Ensembl IDs for shared targets (replace with your actual Ensembl IDs)
shared_target_ensembl_ids = ['ENSG00000164109', 'ENSG00000139618', 'ENSG00000204498']

# Dictionary to store pathways associated with each target
target_pathways = {}

# Retrieve pathways for each target
for ensembl_id in shared_target_ensembl_ids:
    pathways = get_pathways(ensembl_id)
    target_pathways[ensembl_id] = pathways
    print(f"Pathways associated with {ensembl_id}: {pathways}")

# Identify common pathways among the targets
common_pathways = set.intersection(*[set(pathways) for pathways in target_pathways.values()])

print("Common Pathways:", common_pathways)

Error: {"code":404,"reason":"NOT_FOUND","url":"http://reactome.org/ContentService/data/pathways/low/entity/ENSG00000164109","messages":["No result for ENSG00000164109 in null"],"targets":null}
Pathways associated with ENSG00000164109: []
Error: {"code":404,"reason":"NOT_FOUND","url":"http://reactome.org/ContentService/data/pathways/low/entity/ENSG00000139618","messages":["No result for ENSG00000139618 in null"],"targets":null}
Pathways associated with ENSG00000139618: []
Error: {"code":404,"reason":"NOT_FOUND","url":"http://reactome.org/ContentService/data/pathways/low/entity/ENSG00000204498","messages":["No result for ENSG00000204498 in null"],"targets":null}
Pathways associated with ENSG00000204498: []
Common Pathways: set()


In [ ]:
import requests

# Function to retrieve pathways associated with a given gene (Ensembl ID)
def get_pathways(ensembl_id):
    # Construct the URL for KEGG pathway analysis
    url = f"http://rest.kegg.jp/link/pathway/{ensembl_id}"
    # Send GET request to KEGG API
    response = requests.get(url)
    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response
        pathways = set()
        for line in response.text.split('\n'):
            if line:
                fields = line.split('\t')
                pathway_id = fields[1].split(':')[-1]
                pathways.add(pathway_id)
        return pathways
    else:
        print(f"Error retrieving pathways for {ensembl_id}: {response.text}")
        return set()

# Example Ensembl IDs for shared targets (replace with your actual Ensembl IDs)
shared_target_ensembl_ids = ['ENSG00000164109', 'ENSG00000139618', 'ENSG00000204498']

# Dictionary to store pathways associated with each target
target_pathways = {}

# Retrieve pathways for each target
for ensembl_id in shared_target_ensembl_ids:
    pathways = get_pathways(ensembl_id)
    target_pathways[ensembl_id] = pathways
    print(f"Pathways associated with {ensembl_id}: {pathways}")

# Identify common pathways among the targets
common_pathways = set.intersection(*[set(pathways) for pathways in target_pathways.values()])

print("Common Pathways:", common_pathways)

Error retrieving pathways for ENSG00000164109: 
Pathways associated with ENSG00000164109: set()
Error retrieving pathways for ENSG00000139618: 
Pathways associated with ENSG00000139618: set()
Error retrieving pathways for ENSG00000204498: 
Pathways associated with ENSG00000204498: set()
Common Pathways: set()


# structural analysis

To perform structural analysis and examine the three-dimensional structures of the identified targets and the binding sites of the screening compounds, you would typically follow these steps:

Retrieve protein structures: Obtain the three-dimensional structures of the identified targets from databases such as the Protein Data Bank (PDB) or from homology modeling tools.

Analyze binding sites: Identify the binding sites or active sites of the target proteins where the screening compounds are likely to interact. This can be done by analyzing the protein structures or using computational tools for binding site prediction.

Docking studies: Perform computational docking studies to predict the binding affinities and modes of interaction between the screening compounds and the target proteins. This involves simulating the binding of the compounds to the protein structures and predicting the most energetically favorable binding conformations.

Here's a general outline of how you can implement these steps using Python and relevant libraries:

Retrieve protein structures:

You can use libraries like Biopython to fetch protein structures from the Protein Data Bank (PDB) or use tools like SWISS-MODEL for homology modeling if the structure is not available.
Analyze binding sites:

Use tools like PyMOL or ChimeraX to visualize the protein structures and identify the binding sites.
Alternatively, use computational tools like SiteMap or fpocket for automated binding site prediction.
Docking studies:

Utilize molecular docking software such as AutoDock, AutoDock Vina, or GOLD for performing docking studies.
Prepare the ligand (screening compound) structures and protein structures for docking using appropriate file formats.
Perform docking simulations to predict the binding affinities and modes of interaction between the compounds and the protein targets.
Analyze the docking results to identify potential interactions and prioritize compounds for further experimental validation.

In [ ]:
# Example code for structural analysis and docking studies

# Step 1: Retrieve protein structures (example using Biopython)
from Bio.PDB import PDBList

# Function to download PDB file for a given PDB ID
def download_pdb(pdb_id):
    pdb_list = PDBList()
    pdb_list.retrieve_pdb_file(pdb_id, file_format="pdb", pdir=".")

# Example usage:
download_pdb("1abc")  # Replace "1abc" with the actual PDB ID of the protein

# Step 2: Analyze binding sites (manual or using computational tools)
# Visualization of protein structures and binding sites can be done using PyMOL, ChimeraX, etc.

# Step 3: Docking studies (using molecular docking software)
# Example: perform docking using AutoDock Vina (assuming you have the software installed)
# You would typically prepare ligand and protein structures and run docking simulations

ModuleNotFoundError: No module named 'Bio'

 # functional assays

 To conduct functional assays to validate the effects of the screening compounds on the identified targets, you would typically follow these steps:

Selection of Assay: Choose appropriate functional assays based on the targets and compounds of interest. This could include in vitro assays to assess protein activity, cellular assays to evaluate phenotypic changes, or animal models to study therapeutic efficacy in vivo.

Assay Development: Develop or acquire the necessary reagents, cell lines, and protocols for conducting the chosen assays. Optimize assay conditions to ensure robust and reproducible results.

Compound Testing: Perform the functional assays using the screening compounds to evaluate their effects on the targets. This involves treating cells or animals with the compounds and measuring relevant endpoints, such as protein activity, phenotypic changes, or therapeutic outcomes.

Data Analysis: Analyze the assay results to assess the impact of the compounds on the targets. Compare the effects of the compounds to appropriate controls and quantify the observed changes.

Validation and Follow-Up: Validate the assay results through replication and additional experiments, if necessary. Follow up on promising findings with further investigation or optimization.

In [ ]:
# Example code for conducting functional assays

# Step 1: Select appropriate assays based on targets and compounds of interest

# Step 2: Develop or acquire assay protocols and reagents

# Step 3: Perform functional assays
# Example: Conduct cellular assays using appropriate cell lines and endpoints
# Example code for a hypothetical cellular assay using a cell viability endpoint:

def perform_cellular_assay(compound_id, cell_line):
    # Treat cells with the screening compound
    # Incubate cells under appropriate conditions
    # Measure cell viability using a suitable assay (e.g., MTT assay, cell counting)
    # Compare viability of treated cells to untreated controls
    # Analyze assay results

# Perform cellular assays for each compound and target combination
for compound_id, compound_info in similar_compounds_dict.items():
    for target in shared_targets:
        cell_line = "CHO-K1"  # Example cell line
        assay_results = perform_cellular_assay(compound_id, cell_line)
        print(f"Assay results for compound {compound_id} on target {target}: {assay_results}")

# Step 4: Analyze assay results
# Example: Calculate cell viability percentages and compare to controls

# Step 5: Validate and follow up on promising findings
# Example: Confirm results with replicate experiments or additional assays

 #lead optimization strategies, you would typically follow these steps:

Identify Promising Compounds: Based on the results of screening and target identification, select the most promising compounds for lead optimization. These compounds should exhibit desirable activity against the target and have the potential for further improvement.

Analyze Structure-Activity Relationship (SAR): Investigate the structure-activity relationship of the selected compounds to understand how changes in chemical structure affect biological activity. Identify key structural features responsible for activity and explore modifications to enhance potency, selectivity, and other drug-like properties.

Medicinal Chemistry Design: Employ medicinal chemistry principles to design analogs or derivatives of the lead compounds with improved properties. This could involve modifying functional groups, altering stereochemistry, or optimizing physicochemical properties to enhance drug-likeness.

Synthesis and Testing: Synthesize the designed compounds and test them in relevant assays to evaluate their activity, selectivity, and pharmacokinetic properties. Iteratively refine compound designs based on assay results and SAR analysis.

Iterative Optimization: Continuously iterate through the design, synthesis, and testing cycle to refine lead compounds further. Prioritize compounds that demonstrate superior activity, selectivity, and pharmacokinetic profiles for further optimization.

Lead Candidate Selection: Identify lead candidates with optimal properties for preclinical development. Conduct comprehensive characterization and profiling of lead compounds to assess their potential as drug candidates.

Preclinical Evaluation: Evaluate lead candidates in preclinical studies to assess safety, efficacy, and pharmacokinetics in relevant disease models. Select lead candidates with the best overall profile for advancement into clinical development.

In [ ]:
# Example code for lead optimization

# Step 1: Identify promising compounds for lead optimization
promising_compounds = []  # Placeholder for selected compounds

# Step 2: Analyze structure-activity relationship (SAR)
# Perform SAR analysis to identify key structural features related to activity

# Step 3: Medicinal chemistry design
# Design analogs or derivatives of lead compounds with improved properties
# Example: Modify functional groups, optimize physicochemical properties

# Step 4: Synthesis and testing
# Synthesize designed compounds and test them in relevant assays
# Example: Assess activity, selectivity, and pharmacokinetic properties

# Step 5: Iterative optimization
# Refine compound designs based on assay results and SAR analysis
# Example: Iteratively modify compounds and retest them in assays

# Step 6: Lead candidate selection
# Prioritize compounds with optimal properties for preclinical development
# Example: Comprehensive characterization and profiling of lead candidates

# Step 7: Preclinical evaluation
# Evaluate lead candidates in preclinical studies to assess safety, efficacy, and pharmacokinetics
# Example: Conduct in vivo studies in relevant disease models

# Additional considerations:
# - Collaborate with medicinal chemists and computational chemists for compound design and optimization
# - Utilize computational tools for structure-based drug design, virtual screening, and molecular modeling
# - Employ predictive models and machine learning algorithms to guide compound optimization

# Ientify novel therapeutic candidates for DYT1 dystonia

Retrieve Known Compounds: Use ChEMBL to retrieve known compounds with activity against relevant targets associated with DYT1 dystonia.

Perform Similarity Search: Conduct a chemical similarity search using the retrieved compounds as queries. This search will identify compounds with similar chemical structures to known active compounds.

Evaluate Similar Compounds: Assess the similarity search results to identify novel compounds that are structurally similar to known active compounds but may not yet be associated with DYT1 dystonia.

Prioritize Compounds: Prioritize the identified compounds based on various factors such as predicted activity, drug-likeness, and availability for further experimental validation.

In [ ]:
from chembl_webresource_client.new_client import new_client

# Function to retrieve known compounds associated with DYT1 dystonia
def retrieve_known_compounds():
    # Define targets associated with DYT1 dystonia (e.g., TOR1A)
    target_id = 'CHEMBL1829'  # Example: TOR1A target ID

    # Retrieve known compounds associated with the target
    target_client = new_client.target
    compounds = target_client.molecule_chembl_ids(target_id)
    return compounds

# Function to perform chemical similarity search
def perform_similarity_search(compounds, threshold=60):
    similarity_client = new_client.similarity
    similar_compounds_dict = {}

    for compound_id in compounds:
        # Perform similarity search using ChEMBL's similarity search API
        res = similarity_client.filter(smiles=compound_id, similarity=threshold).only(['molecule_chembl_id', 'similarity'])
        similar_compounds = [(entry['molecule_chembl_id'], round(float(entry['similarity']), 2)) for entry in res]
        similar_compounds_dict[compound_id] = similar_compounds

    return similar_compounds_dict

# Step 1: Retrieve known compounds associated with DYT1 dystonia
known_compounds = retrieve_known_compounds()

# Step 2: Perform chemical similarity search
similar_compounds_dict = perform_similarity_search(known_compounds)

# Step 3: Evaluate and prioritize similar compounds
# Example: Identify novel compounds with high similarity to known active compounds

# Step 4: Further analysis and experimental validation of prioritized compounds
# Example: Predict activity, drug-likeness, and experimental validation